Thx to [deepanwayx](https://github.com/deepanwayx/char-and-word-rnn-keras/blob/master/Word%20Language%20Modelling%20LSTM%20.ipynb)

In [1]:
import re
import sys
import string
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, LSTM
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [3]:
rawtext = open(r'F:/data/wonderland.txt','r').read().split('\n')
rawtext = ' '.join(rawtext)
rawtext = [word.strip(string.punctuation) for word in rawtext.split()] #string.punctuation包含所有标点的字符串
rawtext = ' '.join(rawtext)
rawtext = rawtext.replace('-', ' ')
rawtext = ' '.join(rawtext.split())

In [5]:
all_words = rawtext.split()
unique_words = sorted(list(set(all_words)))
n_vocab = len(unique_words)
print("Total Vocab:", n_vocab)
word_to_int = dict((w, i) for i, w in enumerate(unique_words))
int_to_word = dict((i, w) for i, w in enumerate(unique_words))

Total Vocab: 3063


In [7]:
raw_text = rawtext.split()
n_words = len(raw_text)
print(n_words)

26694


In [41]:
#定义输入 输出 并数值化
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_words - seq_length):
    seq_in  = raw_text[i: i+seq_length]
    seq_out = raw_text[i+seq_length]
    dataX.append([word_to_int[word] for word in seq_in])
    dataY.append(word_to_int[seq_out])
n_patterns = len(dataX)
print('Total patterns:', n_patterns)

Total patterns: 26594


In [43]:
# Reshape dataX to size of [samples, time steps, features] and scale it to 0-1
# Represent dataY as one hot encoding
X_train = np.reshape(dataX, (n_patterns, seq_length, 1))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [12]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3063)              787191    
Total params: 1,051,383
Trainable params: 1,051,383
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
# define the checkpoint
filepath="word-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [27]:
model.fit(X_train, Y_train, epochs=10, batch_size=128, callbacks=callbacks_list)

Epoch 1/10
26594/26594 [==============================] - 268s - loss: 6.3347   
Epoch 2/10
26594/26594 [==============================] - 267s - loss: 6.2683   
Epoch 3/10
26594/26594 [==============================] - 264s - loss: 6.2530   
Epoch 4/10
26594/26594 [==============================] - 274s - loss: 6.2453   
Epoch 5/10
26594/26594 [==============================] - 263s - loss: 6.2426   
Epoch 6/10
26594/26594 [==============================] - 266s - loss: 6.2389   
Epoch 7/10
26594/26594 [==============================] - 269s - loss: 6.2329   
Epoch 8/10
26594/26594 [==============================] - 282s - loss: 6.2176   
Epoch 9/10
26594/26594 [==============================] - 278s - loss: 6.1903   
Epoch 10/10
26594/26594 [==============================] - 277s - loss: 6.1574   


In [31]:
# load the network weights
filename = "word-weights-improvement-09-6.1574.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [32]:
model.fit(X_train, Y_train, epochs=40, batch_size=32, callbacks=callbacks_list)

Epoch 1/40
26594/26594 [==============================] - 387s - loss: 6.2680   
Epoch 2/40
26594/26594 [==============================] - 393s - loss: 6.1646   
Epoch 3/40
26594/26594 [==============================] - 380s - loss: 6.0983   
Epoch 4/40
26594/26594 [==============================] - 359s - loss: 6.0299   
Epoch 5/40
26594/26594 [==============================] - 363s - loss: 5.9536   
Epoch 6/40
26594/26594 [==============================] - 333s - loss: 5.8712   
Epoch 7/40
26594/26594 [==============================] - 331s - loss: 5.7634   
Epoch 8/40
26594/26594 [==============================] - 331s - loss: 5.6399   
Epoch 9/40
26594/26594 [==============================] - 341s - loss: 5.5049   
Epoch 10/40
26594/26594 [==============================] - 330s - loss: 5.3569   
Epoch 11/40
26594/26594 [==============================] - 331s - loss: 5.1925   
Epoch 12/40
26594/26594 [==============================] - 339s - loss: 5.0185   
Epoch 13/40
26594/26594 [

In [38]:
# load the network weights
filename = "word-weights-improvement-39-2.0720.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [39]:
start = np.random.randint(0, len(X_train)-1)
pattern = dataX[start]
result = []
print("Seed:")
print("\"", ' '.join([int_to_word[value] for value in pattern]), "\"")
for i in range(200):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x)
    index = np.argmax(prediction)
    result.append(int_to_word[index])
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nGenerated Sequence:")
print(' '.join(result))
print("\nDone."  )

Seed:
" Queen ordering off her unfortunate guests to execution once more the pig baby was sneezing on the Duchess's knee while plates and dishes crashed around it once more the shriek of the Gryphon the squeaking of the Lizard's slate pencil and the choking of the suppressed guinea pigs filled the air mixed up with the distant sobs of the miserable Mock Turtle So she sat on with closed eyes and half believed herself in Wonderland though she knew she had but to open them again and all would change to dull reality the grass would be only rustling in the "

Generated Sequence:
wind and the pool rippling to the waving of the reeds of rattling teacups and made of of upon to herself was be a left shelves to to be that to and it curious when to be that since to King two as a could you no try try the the the be winter for sharp as little a she she it the the the the be ever out of England the hookah that be otherwise of Alice the no Rabbit in no wind and was chance to lying and when as crowded